In [3]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
from utils import *
from modelADT import ModelADT
from Generators import *
from joint_model import Enc_2Dec_Network
from joint_model import Enc_Dec_Network
from params import *
import trainer

import os
import torch
import h5py
import scipy.io
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [78]:
#######################Custom DataLoader########################
# Demands only a few Gigabytes of memory
# `features_wo_offset`: targets for the consistency decoder
# `features_w_offset` : inputs for the network/encoder
# `labels_gaussian_2d`: targets for the location decoder
class DLocDataset(Dataset):
    """DLoc dataset."""

    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dirs (list of strings): List of Directories with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        print(self.root_dir)
        self.transform = transform
        self.n_files = []
        self.file_names = []
        for n,i in enumerate(self.root_dir):
            onlyfiles = next(os.walk(i))[2] #dir is your directory path as string
            self.file_names.append([onlyfiles])
            self.n_files.append(len(onlyfiles))

    def __len__(self):
        return np.sum(self.n_files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        file_num = 0
        count = 0
        for dir_id, n_file in enumerate(self.n_files):
            if idx[0] > (count+n_file):
                count += n_file
                continue
            elif idx[-1] <= (count+n_file-1):
                for i in idx:
                    fname = self.file_names[dir_id][0][i-count]
                    filename = os.path.join(self.root_dir[dir_id], fname)
                    print(filename)
                    f = h5py.File(filename,'r')
                    f_wo_offset = np.transpose(np.array(f.get('features_wo_offset'), dtype=np.float32))
                    f_w_offset = np.transpose(np.array(f.get('features_w_offset'), dtype=np.float32))
                    labels_2d = np.transpose(np.array(f.get('labels_gaussian_2d'), dtype=np.float32))
                    
                    if ( file_num == 0) :
                        features_wo_offset = f_wo_offset
                        features_w_offset = f_w_offset
                        labels_gaussian_2d = labels_2d
                        file_num += 1
                    else:
                        features_wo_offset = np.concatenate((features_wo_offset, f_wo_offset),axis=0)
                        features_w_offset = np.concatenate((features_w_offset, f_w_offset),axis=0)
                        labels_gaussian_2d = np.concatenate((labels_gaussian_2d, labels_2d),axis=0)
                    print(np.shape(labels_gaussian_2d))
                    print(np.shape(features_wo_offset))
                    print(np.shape(features_w_offset))
                count += n_file
                break
            elif idx[-1] > (count+n_file-1):
                for i in range(idx[0],count+n_file-1):
#                     fname = str(i-count+1)+'.h5'
                    fname = self.file_names[dir_id][0][i-count]
                    filename = os.path.join(self.root_dir[dir_id], fname)
                    print(filename)
                    f = h5py.File(filename,'r')
                    f_wo_offset = np.transpose(np.array(f.get('features_wo_offset'), dtype=np.float32))
                    f_w_offset = np.transpose(np.array(f.get('features_w_offset'), dtype=np.float32))
                    labels_2d = np.transpose(np.array(f.get('labels_gaussian_2d'), dtype=np.float32))
                    
                    if ( file_num == 0) :
                        features_wo_offset = f_wo_offset
                        features_w_offset = f_w_offset
                        labels_gaussian_2d = labels_2d
                        file_num += 1
                    else:
                        features_wo_offset = np.concatenate((features_wo_offset, f_wo_offset),axis=0)
                        features_w_offset = np.concatenate((features_w_offset, f_w_offset),axis=0)
                        labels_gaussian_2d = np.concatenate((labels_gaussian_2d, labels_2d),axis=0)
                count += n_file
                idx = np.array(idx)
                idx = idx[np.array(idx)>=count].tolist()
        features_wo_offset = np.squeeze(features_wo_offset)
        features_w_offset = np.squeeze(features_w_offset)
        if(len(np.shape(labels_gaussian_2d))==3):
            if(np.shape(labels_gaussian_2d)[0]!=1):
                labels_gaussian_2d = np.expand_dims(labels_gaussian_2d, axis=1)

        sample = {'features_wo_offset': features_wo_offset, 'features_w_offset': features_w_offset, 'labels_gaussian_2d': labels_gaussian_2d}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [79]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        features_wo_offset, features_w_offset, labels_gaussian_2d = sample['features_wo_offset'], sample['features_w_offset'], sample['labels_gaussian_2d']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        return {'labels_gaussian_2d': torch.from_numpy(labels_gaussian_2d),
                'features_wo_offset': torch.from_numpy(features_wo_offset),
                'features_w_offset': torch.from_numpy(features_w_offset)}

In [80]:
trainpath = ['/media/datadisk/Roshan/datasets/DLoc_sim_data/features/jacobs_July28/',
             '/media/datadisk/Roshan/datasets/DLoc_sim_data/features/jacobs_July28_2/']



train_data = DLocDataset(root_dir=trainpath,
                                  transform=transforms.Compose([
                                  ToTensor()
                                  ]))
train_loader =torch.utils.data.DataLoader(train_data, batch_size=opt_exp.batch_size, shuffle=True)

['/media/datadisk/Roshan/datasets/DLoc_sim_data/features/jacobs_July28/', '/media/datadisk/Roshan/datasets/DLoc_sim_data/features/jacobs_July28_2/']


In [81]:
print(np.shape(train_data.file_names))
# print(train_data.file_names[2][0][2])
# print(np.shape(train_data.file_names[0]))
# print(np.shape(train_data.file_names[1]))
# print(np.shape(train_data.file_names[2]))
print(train_data.n_files)
print(len(train_data))

(2, 1)
[11440, 14732]
26172


In [82]:
print(train_data)

In [84]:
sample = train_data[[1]]

print('Length of training data = %d' % len(train_data))
print(f"Size of features wo offset {sample['features_wo_offset'].size()}")
print(f"Size of features w offset  {sample['features_w_offset'].size()}")
print(f"Size of labels             {sample['labels_gaussian_2d'].size()}")

/media/datadisk/Roshan/datasets/DLoc_sim_data/features/jacobs_July28/10.h5
(1, 161, 361)
(1, 4, 161, 361)
(1, 4, 161, 361)
Length of training data = 26172
Size of features wo offset torch.Size([4, 161, 361])
Size of features w offset  torch.Size([4, 161, 361])
Size of labels             torch.Size([1, 161, 361])


In [15]:
print(train_data.file_names[1][0][0])

1.h5


In [16]:
print(len(train_loader))

818


In [17]:
print(28724/32)

897.625
